In [7]:
import requests  # 导入requests库用于发送HTTP请求
from lxml import etree  # 导入etree用于解析HTML
import time  # 导入time库用于添加延时

# 设置请求头，模拟浏览器访问，避免被反爬
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0',
    'Cookie':'bid=Sq8nsxTl7sw; douban-fav-remind=1; _pk_id.100001.4cf6=946f699214d356e4.1714896880.; __gads=ID=b15bf09146bcb2e6:T=1714896881:RT=1714896881:S=ALNI_MbOkdAlJA9MRRTmeStzIlq4UEk8ng; __gpi=UID=00000e0c862706ca:T=1714896881:RT=1714896881:S=ALNI_Mbame5e40ZVYRlvHvsi0phlJQcBiA; _pk_ses.100001.4cf6=1; __utma=30149280.1446488683.1705427805.1714896880.1732241206.3; __utmb=30149280.0.10.1732241206; __utmc=30149280; __utmz=30149280.1732241206.3.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utma=223695111.1994243429.1714896880.1714896880.1732241206.2; __utmb=223695111.0.10.1732241206; __utmc=223695111; __utmz=223695111.1732241206.2.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __yadk_uid=778KcdbZkmlMaiCICmarNWjHFP1feDZl; dbcl2="180130411:e7Yv9RDJjjU"; ck=z-re; push_noty_num=0; push_doumail_num=0; ll="108304"; frodotk_db="694605c187345c353bd21144959d149f"; _vwo_uuid_v2=D5B3ED703FA2FCE0BBF31BF7EAF8E866D|09d62378902d028b13e127e0473df160; ap_v=0,6.0'
}

# 创建列表存储所有电影的URL
movie_urls = []

# 豆瓣电影 TOP250 一页 25 部电影，需要遍历 10 页来获取所有250部电影
for page in range(10):  # 循环10次，每次获取25部电影的信息
    # 构造每一页的URL，使用f-string格式化字符串
    url = f'https://movie.douban.com/top250?start={page * 25}&filter='
    
    # 使用设置好的headers发送GET请求获取页面内容
    response = requests.get(url, headers=headers)
    
    # 添加延时，避免请求过快被豆瓣封IP
    time.sleep(1)  # 每次请求后等待1秒
    
    # 使用etree.HTML()解析返回的HTML内容
    html = etree.HTML(response.text)
    
    # 使用XPath提取电影链接：
    # //div[@class="hd"] 选择class为hd的div元素
    # /a 选择其下的a标签
    # /@href 获取a标签的href属性值（即电影详情页URL）
    urls = html.xpath('//div[@class="hd"]/a/@href')
    
    # 将当前页面获取到的所有电影URL添加到movie_urls列表中
    movie_urls.extend(urls)

# 打印所有获取到的电影URL
# enumerate(movie_urls, 1)从1开始给每个URL编号
for i, url in enumerate(movie_urls, 1):
    print(f"{i}. {url}")

1. https://movie.douban.com/subject/1292052/
2. https://movie.douban.com/subject/1291546/
3. https://movie.douban.com/subject/1292720/
4. https://movie.douban.com/subject/1292722/
5. https://movie.douban.com/subject/1291561/
6. https://movie.douban.com/subject/1292063/
7. https://movie.douban.com/subject/1295644/
8. https://movie.douban.com/subject/1889243/
9. https://movie.douban.com/subject/3541415/
10. https://movie.douban.com/subject/1292064/
11. https://movie.douban.com/subject/1295124/
12. https://movie.douban.com/subject/3011091/
13. https://movie.douban.com/subject/1292001/
14. https://movie.douban.com/subject/3793023/
15. https://movie.douban.com/subject/1291549/
16. https://movie.douban.com/subject/2131459/
17. https://movie.douban.com/subject/25662329/
18. https://movie.douban.com/subject/1307914/
19. https://movie.douban.com/subject/1296141/
20. https://movie.douban.com/subject/1292213/
21. https://movie.douban.com/subject/5912992/
22. https://movie.douban.com/subject/12918

In [5]:
print("学号：222226205227")

学号：222226205227


In [13]:
from pymongo import MongoClient

# 连接MongoDB数据库
client = MongoClient('localhost', 27017)  # 创建MongoDB客户端连接,连接到本地MongoDB服务器
db = client['豆瓣电影榜']  # 选择或创建名为'豆瓣电影榜'的数据库
collection = db['Top250(XPATH)']  # 选择或创建名为'Top250(XPATH)'的集合

# 遍历所有电影URL并获取详细信息
for url in movie_urls:
    # 发送HTTP GET请求获取电影详情页面
    response = requests.get(url, headers=headers)
    time.sleep(0.5)  # 添加0.5秒延迟,避免请求过于频繁
    html = etree.HTML(response.text)  # 解析HTML内容
    
    # 使用XPath提取电影名称
    name = html.xpath('//span[@property="v:itemreviewed"]/text()')[0]
    
    # 使用XPath提取电影类型(可能有多个)
    genres = html.xpath('//span[@property="v:genre"]/text()')
    description = ' '.join(genres)  # 将多个类型用空格连接成字符串
    
    # 使用XPath提取评分
    rating = html.xpath('//strong[@class="ll rating_num"][@property="v:average"]/text()')[0]
    
    # 使用XPath提取评价人数,并添加"人评价"后缀
    votes = html.xpath('//span[@property="v:votes"]/text()')[0] + '人评价'
    
    # 获取剧情简介 - 首先尝试获取完整版简介
    intro = html.xpath('//span[@class="all hidden"]/text()')
    if not intro:  # 如果没有完整版简介(即没有"展开"按钮)
        intro = html.xpath('//span[@property="v:summary"]/text()')  # 获取默认显示的简介
    intro = ''.join(intro).strip()  # 合并简介文本并去除首尾空白
    
    # 获取获奖情况
    awards_link = html.xpath('//div[@class="mod"]//a[contains(text(), "全部")]/@href')
    awards = []
    
    if awards_link:  # 如果存在"全部"链接(说明奖项数量较多)
        # 访问获奖详情页面
        awards_response = requests.get(awards_link[0], headers=headers)
        time.sleep(0.5)  # 添加延迟
        awards_html = etree.HTML(awards_response.text)
        # 提取所有奖项名称
        awards = awards_html.xpath('//a[contains(@href, "/awards/")]/text()')
    else:  # 如果没有"全部"链接,直接从当前页面获取奖项
        awards = html.xpath('//div[@class="mod"]//a[contains(@href, "/awards/")]/text()')
    
    # 过滤奖项,只保留包含"届"字的奖项名称
    awards = [award for award in awards if '届' in award]
    awards = awards if awards else '0'  # 如果没有获奖记录,则标记为'0'
    
    # 构建要插入数据库的文档
    movie_doc = {
        '名称': name,
        '类型': description,
        '评分': rating,
        '评价': votes,
        '简介': intro,
        '获奖': awards
    }
    
    # 将电影信息插入MongoDB数据库
    collection.insert_one(movie_doc)
    print(f"Successfully saved to MongoDB: {name}")  # 打印成功保存的电影名称

Successfully saved to MongoDB: 肖申克的救赎 The Shawshank Redemption
Successfully saved to MongoDB: 霸王别姬
Successfully saved to MongoDB: 阿甘正传 Forrest Gump
Successfully saved to MongoDB: 泰坦尼克号 Titanic
Successfully saved to MongoDB: 千与千寻 千と千尋の神隠し
Successfully saved to MongoDB: 美丽人生 La vita è bella
Successfully saved to MongoDB: 这个杀手不太冷 Léon
Successfully saved to MongoDB: 星际穿越 Interstellar
Successfully saved to MongoDB: 盗梦空间 Inception
Successfully saved to MongoDB: 楚门的世界 The Truman Show
Successfully saved to MongoDB: 辛德勒的名单 Schindler's List
Successfully saved to MongoDB: 忠犬八公的故事 Hachi: A Dog's Tale
Successfully saved to MongoDB: 海上钢琴师 La leggenda del pianista sull'oceano
Successfully saved to MongoDB: 三傻大闹宝莱坞 3 Idiots
Successfully saved to MongoDB: 放牛班的春天 Les choristes
Successfully saved to MongoDB: 机器人总动员 WALL·E
Successfully saved to MongoDB: 疯狂动物城 Zootopia
Successfully saved to MongoDB: 无间道 無間道
Successfully saved to MongoDB: 控方证人 Witness for the Prosecution
Successfully saved to MongoDB: 大话西游之

In [2]:
import requests
from bs4 import BeautifulSoup
import time

# 设置请求头，模拟浏览器访问，避免被反爬
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0',
    'Cookie':'bid=Sq8nsxTl7sw; douban-fav-remind=1; _pk_id.100001.8cb4=8ce3ec32ddd46ab4.1705427804.; __yadk_uid=dXMiHRHrxB23Wk7ufqD7tmzqIqpCVUkD; __gads=ID=b15bf09146bcb2e6:T=1714896881:RT=1714896881:S=ALNI_MbOkdAlJA9MRRTmeStzIlq4UEk8ng; __gpi=UID=00000e0c862706ca:T=1714896881:RT=1714896881:S=ALNI_Mbame5e40ZVYRlvHvsi0phlJQcBiA; dbcl2="180130411:e7Yv9RDJjjU"; push_noty_num=0; push_doumail_num=0; ll="108304"; _vwo_uuid_v2=D5B3ED703FA2FCE0BBF31BF7EAF8E866D|09d62378902d028b13e127e0473df160; __utmz=30149280.1732247596.4.2.utmcsr=movie.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/subject/1292052/; __utmv=30149280.18013; __utma=30149280.1446488683.1705427805.1732257047.1732373607.6; __utmc=30149280; ck=z-re; frodotk_db="e74ad67faba0a181b5e0837f02e1e483"; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1732374761%2C%22https%3A%2F%2Fmovie.douban.com%2Ftop250%22%5D; _pk_ses.100001.8cb4=1; __utmt=1; __utmb=30149280.2.10.1732373607'
}

# 创建空列表用于存储所有电影的URL
movie_urls = []

# 遍历豆瓣TOP250的10个页面（每页25部电影）
for page in range(10):  # 获取250部电影需要遍历10页
    # 构造每一页的URL，使用f-string格式化字符串
    url = f'https://movie.douban.com/top250?start={page * 25}&filter='
    
    # 使用requests发送GET请求获取页面内容
    response = requests.get(url, headers=headers)
    
    # 添加延时，避免请求过快被豆瓣封IP
    time.sleep(1)  # 每次请求后等待1秒
    
    # 使用BeautifulSoup解析HTML内容
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 查找所有包含电影链接的div元素（class为'hd'的div）
    movie_divs = soup.find_all('div', class_='hd')
    
    # 从每个div中提取电影详情页的URL
    urls = [div.find('a')['href'] for div in movie_divs]
    
    # 将当前页面获取的URL添加到总列表中
    movie_urls.extend(urls)

# 打印所有获取到的电影URL，并标注序号
for i, url in enumerate(movie_urls, 1):
    print(f"{i}. {url}")

1. https://movie.douban.com/subject/1292052/
2. https://movie.douban.com/subject/1291546/
3. https://movie.douban.com/subject/1292720/
4. https://movie.douban.com/subject/1292722/
5. https://movie.douban.com/subject/1291561/
6. https://movie.douban.com/subject/1292063/
7. https://movie.douban.com/subject/1295644/
8. https://movie.douban.com/subject/1889243/
9. https://movie.douban.com/subject/3541415/
10. https://movie.douban.com/subject/1292064/
11. https://movie.douban.com/subject/1295124/
12. https://movie.douban.com/subject/3011091/
13. https://movie.douban.com/subject/1292001/
14. https://movie.douban.com/subject/3793023/
15. https://movie.douban.com/subject/1291549/
16. https://movie.douban.com/subject/2131459/
17. https://movie.douban.com/subject/25662329/
18. https://movie.douban.com/subject/1307914/
19. https://movie.douban.com/subject/1296141/
20. https://movie.douban.com/subject/1292213/
21. https://movie.douban.com/subject/5912992/
22. https://movie.douban.com/subject/12918

In [6]:
print("学号：222226205227")

学号：222226205227


In [4]:
from pymongo import MongoClient

# 连接MongoDB数据库
client = MongoClient('localhost', 27017)  # 创建MongoDB客户端连接,连接到本地MongoDB服务器的27017端口
db = client['豆瓣电影榜']  # 选择或创建名为'豆瓣电影榜'的数据库
collection = db['Top250(BS4)']  # 选择或创建名为'Top250(BS4)'的集合

# 遍历所有电影链接并获取每部电影的详细信息
for url in movie_urls[0:]:
    # 发送HTTP GET请求获取电影详情页面
    response = requests.get(url, headers=headers)
    time.sleep(2)  # 每次请求间隔2秒,避免请求过于频繁
    # 使用BeautifulSoup解析HTML内容
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 获取电影名称 - 查找带有特定property属性的span标签
    name = soup.find('span', property='v:itemreviewed').text
    
    # 获取电影类型 - 查找所有带有genre属性的span标签
    genres = [genre.text for genre in soup.find_all('span', property='v:genre')]
    description = ' '.join(genres)  # 将所有类型用空格连接成一个字符串
    
    # 获取评分 - 查找带有特定class和property的strong标签
    rating = soup.find('strong', {'class': 'll rating_num', 'property': 'v:average'}).text
    
    # 获取评价人数 - 查找带有votes属性的span标签,并添加"人评价"后缀
    votes = soup.find('span', property='v:votes').text + '人评价'
    
    # 获取剧情简介
    intro_span = soup.find('span', class_='all hidden')  # 首先尝试获取完整展开的简介
    if not intro_span:  # 如果没有找到展开的简介
        intro_span = soup.find('span', property='v:summary')  # 获取短简介
    intro = intro_span.text.strip()  # 去除首尾空白字符
    
    # 获取获奖情况
    awards_div = soup.find('div', class_='mod')  # 查找包含获奖信息的div
    awards = []
    
    if awards_div:
        # 查找是否有"全部"链接(表示奖项数超过页面显示限制)
        awards_link = awards_div.find('a', string='全部')
        if awards_link:  # 如果存在"全部"链接
            # 访问完整的获奖页面
            awards_response = requests.get(awards_link['href'], headers=headers)
            time.sleep(0.5)  # 短暂延迟
            awards_soup = BeautifulSoup(awards_response.text, 'html.parser')
            # 提取所有包含"届"字的奖项链接文本
            awards = [a.text for a in awards_soup.find_all('a', href=lambda x: x and '/awards/' in x) if '届' in a.text]
        else:  # 如果没有"全部"链接,直接从当前页面获取奖项
            awards = [a.text for a in awards_div.find_all('a', href=lambda x: x and '/awards/' in x) if '届' in a.text]
    
    awards = awards if awards else '0'  # 如果没有获得任何奖项,则标记为'0'
    
    # 构建包含电影信息的文档
    movie_doc = {
        '名称': name,
        '类型': description,
        '评分': rating,
        '评价': votes,
        '简介': intro,
        '获奖': awards
    }
    
    # 将电影文档插入MongoDB集合
    collection.insert_one(movie_doc)
    print(f"Successfully saved to MongoDB: {name}")  # 打印成功保存的电影名称

Successfully saved to MongoDB: 肖申克的救赎 The Shawshank Redemption
Successfully saved to MongoDB: 霸王别姬
Successfully saved to MongoDB: 阿甘正传 Forrest Gump
Successfully saved to MongoDB: 泰坦尼克号 Titanic
Successfully saved to MongoDB: 千与千寻 千と千尋の神隠し
Successfully saved to MongoDB: 美丽人生 La vita è bella
Successfully saved to MongoDB: 这个杀手不太冷 Léon
Successfully saved to MongoDB: 星际穿越 Interstellar
Successfully saved to MongoDB: 盗梦空间 Inception
Successfully saved to MongoDB: 楚门的世界 The Truman Show
Successfully saved to MongoDB: 辛德勒的名单 Schindler's List
Successfully saved to MongoDB: 忠犬八公的故事 Hachi: A Dog's Tale
Successfully saved to MongoDB: 海上钢琴师 La leggenda del pianista sull'oceano
Successfully saved to MongoDB: 三傻大闹宝莱坞 3 Idiots
Successfully saved to MongoDB: 放牛班的春天 Les choristes
Successfully saved to MongoDB: 机器人总动员 WALL·E
Successfully saved to MongoDB: 疯狂动物城 Zootopia
Successfully saved to MongoDB: 无间道 無間道
Successfully saved to MongoDB: 控方证人 Witness for the Prosecution
Successfully saved to MongoDB: 大话西游之